In [32]:
# Import useful libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Ignore warnings
import warnings
warnings.simplefilter('ignore')

# Import evaluation libraries
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Random seed for reproducibility
seed = 202
np.random.seed(seed)

In [33]:
# Import the train set and test set
train_data = pd.read_csv("cat_train.csv", delimiter=",")
test_data = pd.read_csv("cat_test.csv", delimiter=",")

# Drop the ID column
train_data = train_data.drop('ID', axis=1)
train_data = train_data.drop('Regions_Cluster', axis=1)
# Sort the dataset
train_data = train_data.iloc[np.random.permutation(len(train_data))]
train_data = train_data.reset_index(drop=True)

train_data.fillna(999999, inplace=True)
test_data.fillna(999999, inplace=True)

In [34]:
array = np.asarray(train_data)

# X,Y are the splits between features and labels used to evaluate SelectKBest
X = array[:,0:train_data.shape[1]-1]
X = np.asarray(X)
Y = array[:,train_data.shape[1]-1]
Y = np.asarray(Y)

# Evaluate the features with a chi2 test by using SelectKBest
# It must be at most max_feature_number - 1
feature_number = 29

chi2_test = SelectKBest(score_func=chi2, k=feature_number)
fit = chi2_test.fit(X,Y)

In [35]:
def features_labels_split(data):
    
    features = data[train_data.columns[chi2_test.get_support(indices=True)]]
    labels = data['Product']
    return features, labels

In [36]:
def standardize_features(features):

    scaler = MinMaxScaler().fit(features)
    features = scaler.transform(features)
    return features

In [37]:
def train_test_validation_split(features, labels, val_samples, test_samples):

    features = np.asarray(features)
    features = standardize_features(features)
    labels =np.asarray(labels)
    
    X_test = features[0:test_samples]
    y_test = labels[0:test_samples]

    X_val = features[test_samples:test_samples + val_samples]
    y_val = labels[test_samples:test_samples + val_samples]

    X_train = features[test_samples + val_samples:]
    y_train = labels[test_samples + val_samples:]

    return X_train, X_test, X_val, y_train, y_test, y_val

In [38]:
import keras
from keras.models import Sequential
from keras.layers import Activation
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam
from keras import backend as K
from keras.callbacks import EarlyStopping, TensorBoard
import tensorflow as tf

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [39]:
# Import the train set and test set
train_data = pd.read_csv("cat_train.csv", delimiter=",")
test_data = pd.read_csv("cat_test.csv", delimiter=",")

# Drop the ID column
train_data = train_data.drop('ID', axis=1)
train_data = train_data.drop('Regions_Cluster', axis=1)
# Sort the dataset
train_data = train_data.iloc[np.random.permutation(len(train_data))]
train_data = train_data.reset_index(drop=True)

train_data.fillna(-999, inplace=True)
test_data.fillna(-999, inplace=True)

In [40]:
train_feat, train_label = features_labels_split(train_data)
train_label = keras.utils.to_categorical(train_label, 4)

num_val_samples = 950
num_test_samples = 950
X_train, X_test, X_val, y_train, y_test, y_val = train_test_validation_split(train_feat, train_label, num_val_samples, num_test_samples)

In [41]:
class TrainValTensorBoard(TensorBoard):
    def __init__(self, log_dir='./logs', **kwargs):
        # Make the original `TensorBoard` log to a subdirectory 'training'
        training_log_dir = os.path.join(log_dir, 'training')
        super(TrainValTensorBoard, self).__init__(training_log_dir, **kwargs)

        # Log the validation metrics to a separate subdirectory
        self.val_log_dir = os.path.join(log_dir, 'validation')

    def set_model(self, model):
        # Setup writer for validation metrics
        self.val_writer = tf.summary.FileWriter(self.val_log_dir)
        super(TrainValTensorBoard, self).set_model(model)

    def on_epoch_end(self, epoch, logs=None):
        # Pop the validation logs and handle them separately with
        # `self.val_writer`. Also rename the keys so that they can
        # be plotted on the same figure with the training metrics
        logs = logs or {}
        val_logs = {k.replace('val_', ''): v for k, v in logs.items() if k.startswith('val_')}
        for name, value in val_logs.items():
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value.item()
            summary_value.tag = name
            self.val_writer.add_summary(summary, epoch)
        self.val_writer.flush()

        # Pass the remaining logs to `TensorBoard.on_epoch_end`
        logs = {k: v for k, v in logs.items() if not k.startswith('val_')}
        super(TrainValTensorBoard, self).on_epoch_end(epoch, logs)

    def on_train_end(self, logs=None):
        super(TrainValTensorBoard, self).on_train_end(logs)
        self.val_writer.close()

In [45]:
tensorboard = TensorBoard(log_dir="C:/Users/eugen/Documents/GitHub/Vodafone_Challenge/Eugenio/logs", histogram_freq=20, write_graph=True, write_images=False)

neurons = 64
dropout = 0.3
batch_size = 64         
epochs = 140


output_activation_function = 'softmax'
activation_function = 'relu'

loss = 'categorical_crossentropy'
learning_rate = 0.0025
optimizer= Adam(learning_rate)

def build_model(inputs, output_size, neurons, activ_func=activation_function,
                dropout = dropout, loss=loss, optimizer=optimizer):
    
    model = Sequential()
    model.add(Dense(neurons, input_shape=(X_train.shape[1],)))
    model.add(Activation(activation_function))
    model.add(Dropout(dropout))
    model.add(Dense(neurons))
    model.add(Activation(activation_function))
    model.add(Dense(units=output_size, activation=output_activation_function))    
    model.compile(loss=loss, optimizer=optimizer, metrics=[f1])
    return model

In [46]:
# initialise model architecture
output_size = 4

model = build_model(X_train, output_size=output_size, neurons=neurons)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 64)                1920      
_________________________________________________________________
activation_11 (Activation)   (None, 64)                0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 64)                4160      
_________________________________________________________________
activation_12 (Activation)   (None, 64)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 4)                 260       
Total params: 6,340
Trainable params: 6,340
Non-trainable params: 0
_________________________________________________________________


In [44]:
# train model on data
history = model.fit(X_train, y_train,
                            epochs=epochs, batch_size=batch_size,
                            verbose=1, validation_data=(X_val, y_val),
                             shuffle=False)

Train on 7667 samples, validate on 950 samples
Epoch 1/5000
7667/7667 [==============================] - 1s 190us/step - loss: 1.0938 - f1: 0.5199 - val_loss: 1.0256 - val_f1: 0.5386
Epoch 2/5000
7667/7667 [==============================] - 1s 80us/step - loss: 1.0498 - f1: 0.5450 - val_loss: 1.0067 - val_f1: 0.5411
Epoch 3/5000
7667/7667 [==============================] - 1s 81us/step - loss: 1.0290 - f1: 0.5514 - val_loss: 1.0043 - val_f1: 0.5288
Epoch 4/5000
7667/7667 [==============================] - 1s 82us/step - loss: 1.0217 - f1: 0.5569 - val_loss: 1.0013 - val_f1: 0.5447
Epoch 5/5000
7667/7667 [==============================] - 1s 84us/step - loss: 1.0199 - f1: 0.5564 - val_loss: 0.9998 - val_f1: 0.5241
Epoch 6/5000
7667/7667 [==============================] - 1s 82us/step - loss: 1.0167 - f1: 0.5563 - val_loss: 1.0010 - val_f1: 0.5283
Epoch 7/5000
7667/7667 [==============================] - 1s 83us/step - loss: 1.0145 - f1: 0.5606 - val_loss: 0.9954 - val_f1: 0.5525
Epoch 8

Epoch 61/5000
7667/7667 [==============================] - 1s 86us/step - loss: 0.9637 - f1: 0.5894 - val_loss: 1.0027 - val_f1: 0.5699
Epoch 62/5000
7667/7667 [==============================] - 1s 81us/step - loss: 0.9602 - f1: 0.5905 - val_loss: 1.0083 - val_f1: 0.5713
Epoch 63/5000
7667/7667 [==============================] - 1s 81us/step - loss: 0.9621 - f1: 0.5893 - val_loss: 1.0062 - val_f1: 0.5676
Epoch 64/5000
7667/7667 [==============================] - 1s 84us/step - loss: 0.9626 - f1: 0.5885 - val_loss: 0.9980 - val_f1: 0.5749
Epoch 65/5000
7667/7667 [==============================] - 1s 83us/step - loss: 0.9594 - f1: 0.5913 - val_loss: 1.0072 - val_f1: 0.5676
Epoch 66/5000
7667/7667 [==============================] - 1s 80us/step - loss: 0.9585 - f1: 0.5850 - val_loss: 1.0037 - val_f1: 0.5735
Epoch 67/5000
7667/7667 [==============================] - 1s 85us/step - loss: 0.9558 - f1: 0.5925 - val_loss: 1.0008 - val_f1: 0.5800
Epoch 68/5000
7667/7667 [=======================

7667/7667 [==============================] - 1s 79us/step - loss: 0.9278 - f1: 0.6069 - val_loss: 1.0423 - val_f1: 0.5714
Epoch 122/5000
7667/7667 [==============================] - 1s 79us/step - loss: 0.9271 - f1: 0.6103 - val_loss: 1.0415 - val_f1: 0.5720
Epoch 123/5000
7667/7667 [==============================] - 1s 79us/step - loss: 0.9244 - f1: 0.6122 - val_loss: 1.0500 - val_f1: 0.5750
Epoch 124/5000
7667/7667 [==============================] - 1s 80us/step - loss: 0.9327 - f1: 0.6081 - val_loss: 1.0423 - val_f1: 0.5748
Epoch 125/5000
7667/7667 [==============================] - 1s 80us/step - loss: 0.9280 - f1: 0.6042 - val_loss: 1.0406 - val_f1: 0.5732
Epoch 126/5000
7667/7667 [==============================] - 1s 79us/step - loss: 0.9263 - f1: 0.6117 - val_loss: 1.0425 - val_f1: 0.5750
Epoch 127/5000
7667/7667 [==============================] - 1s 79us/step - loss: 0.9241 - f1: 0.6111 - val_loss: 1.0346 - val_f1: 0.5777
Epoch 128/5000
7667/7667 [==============================

Epoch 181/5000
7667/7667 [==============================] - 1s 94us/step - loss: 0.9111 - f1: 0.6198 - val_loss: 1.0838 - val_f1: 0.5772
Epoch 182/5000
7667/7667 [==============================] - 1s 94us/step - loss: 0.9070 - f1: 0.6224 - val_loss: 1.0754 - val_f1: 0.5754
Epoch 183/5000
7667/7667 [==============================] - 1s 94us/step - loss: 0.9075 - f1: 0.6176 - val_loss: 1.0832 - val_f1: 0.5676
Epoch 184/5000
7667/7667 [==============================] - 1s 94us/step - loss: 0.9071 - f1: 0.6203 - val_loss: 1.0844 - val_f1: 0.5591
Epoch 185/5000
7667/7667 [==============================] - 1s 96us/step - loss: 0.9082 - f1: 0.6156 - val_loss: 1.0815 - val_f1: 0.5623
Epoch 186/5000
7667/7667 [==============================] - 1s 95us/step - loss: 0.9081 - f1: 0.6164 - val_loss: 1.0797 - val_f1: 0.5682
Epoch 187/5000
7667/7667 [==============================] - 1s 92us/step - loss: 0.9063 - f1: 0.6235 - val_loss: 1.0874 - val_f1: 0.5692
Epoch 188/5000
7667/7667 [===============

7667/7667 [==============================] - 1s 95us/step - loss: 0.8974 - f1: 0.6236 - val_loss: 1.1069 - val_f1: 0.5798
Epoch 241/5000
7667/7667 [==============================] - 1s 83us/step - loss: 0.9010 - f1: 0.6244 - val_loss: 1.1119 - val_f1: 0.5669
Epoch 242/5000
7667/7667 [==============================] - 1s 84us/step - loss: 0.8910 - f1: 0.6271 - val_loss: 1.1161 - val_f1: 0.5733
Epoch 243/5000
7667/7667 [==============================] - 1s 86us/step - loss: 0.8924 - f1: 0.6266 - val_loss: 1.1071 - val_f1: 0.5769
Epoch 244/5000
7667/7667 [==============================] - 1s 94us/step - loss: 0.8890 - f1: 0.6286 - val_loss: 1.1106 - val_f1: 0.5798
Epoch 245/5000
7667/7667 [==============================] - 1s 95us/step - loss: 0.8928 - f1: 0.6273 - val_loss: 1.1163 - val_f1: 0.5660
Epoch 246/5000
7667/7667 [==============================] - 1s 95us/step - loss: 0.8840 - f1: 0.6288 - val_loss: 1.1105 - val_f1: 0.5691
Epoch 247/5000
7667/7667 [==============================

Epoch 300/5000
7667/7667 [==============================] - 1s 92us/step - loss: 0.8844 - f1: 0.6304 - val_loss: 1.1376 - val_f1: 0.5650
Epoch 301/5000
7667/7667 [==============================] - 1s 79us/step - loss: 0.8844 - f1: 0.6348 - val_loss: 1.1402 - val_f1: 0.5596
Epoch 302/5000
7667/7667 [==============================] - 1s 78us/step - loss: 0.8771 - f1: 0.6327 - val_loss: 1.1226 - val_f1: 0.5661
Epoch 303/5000
7667/7667 [==============================] - 1s 86us/step - loss: 0.8793 - f1: 0.6333 - val_loss: 1.1346 - val_f1: 0.5637
Epoch 304/5000
7667/7667 [==============================] - 1s 93us/step - loss: 0.8828 - f1: 0.6323 - val_loss: 1.1374 - val_f1: 0.5727
Epoch 305/5000
7667/7667 [==============================] - 1s 93us/step - loss: 0.8791 - f1: 0.6306 - val_loss: 1.1354 - val_f1: 0.5670
Epoch 306/5000
7667/7667 [==============================] - 1s 92us/step - loss: 0.8833 - f1: 0.6310 - val_loss: 1.1528 - val_f1: 0.5633
Epoch 307/5000
7667/7667 [===============

7667/7667 [==============================] - 1s 88us/step - loss: 0.8732 - f1: 0.6378 - val_loss: 1.1588 - val_f1: 0.5628
Epoch 360/5000
7667/7667 [==============================] - 1s 96us/step - loss: 0.8786 - f1: 0.6333 - val_loss: 1.1567 - val_f1: 0.5631
Epoch 361/5000
7667/7667 [==============================] - 1s 94us/step - loss: 0.8834 - f1: 0.6319 - val_loss: 1.1540 - val_f1: 0.5623
Epoch 362/5000
7667/7667 [==============================] - 1s 92us/step - loss: 0.8726 - f1: 0.6366 - val_loss: 1.1488 - val_f1: 0.5722
Epoch 363/5000
7667/7667 [==============================] - 1s 92us/step - loss: 0.8649 - f1: 0.6433 - val_loss: 1.1619 - val_f1: 0.5662
Epoch 364/5000
7667/7667 [==============================] - 1s 80us/step - loss: 0.8772 - f1: 0.6382 - val_loss: 1.1617 - val_f1: 0.5646
Epoch 365/5000
7667/7667 [==============================] - 1s 79us/step - loss: 0.8700 - f1: 0.6354 - val_loss: 1.1645 - val_f1: 0.5619
Epoch 366/5000
7667/7667 [==============================

Epoch 419/5000
7667/7667 [==============================] - 1s 92us/step - loss: 0.8697 - f1: 0.6381 - val_loss: 1.1642 - val_f1: 0.5590
Epoch 420/5000
7667/7667 [==============================] - 1s 95us/step - loss: 0.8663 - f1: 0.6408 - val_loss: 1.1626 - val_f1: 0.5687
Epoch 421/5000
7667/7667 [==============================] - 1s 92us/step - loss: 0.8653 - f1: 0.6428 - val_loss: 1.1794 - val_f1: 0.5485
Epoch 422/5000
7667/7667 [==============================] - 1s 99us/step - loss: 0.8634 - f1: 0.6433 - val_loss: 1.1853 - val_f1: 0.5603
Epoch 423/5000
7667/7667 [==============================] - 1s 95us/step - loss: 0.8662 - f1: 0.6388 - val_loss: 1.1669 - val_f1: 0.5761
Epoch 424/5000
7667/7667 [==============================] - 1s 98us/step - loss: 0.8721 - f1: 0.6398 - val_loss: 1.1796 - val_f1: 0.5645
Epoch 425/5000
7667/7667 [==============================] - 1s 95us/step - loss: 0.8660 - f1: 0.6371 - val_loss: 1.1686 - val_f1: 0.5701
Epoch 426/5000
7667/7667 [===============

KeyboardInterrupt: 